In [5]:
import requests
import time

# Initialization 
API_KEY = {'X-API-key': 'HCA61YVQ'}
s = requests.Session() 
s.headers.update(API_KEY)

resp = s.get('http://localhost:9999/v1/case')  # removed space before get
resp  # 200 means success

<Response [200]>

In [28]:
def get_tick():
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        return resp.json()['tick']
    return 0

def get_position(ticker):
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        for sec in resp.json():
            if sec['ticker'] == ticker:
                return sec['position']
    return 0

def send_order(ticker, action, price, qty):
    params = {
        'ticker': ticker,
        'type': 'LIMIT',
        'quantity': qty,
        'action': action,
        'price': price
    }
    resp = s.post('http://localhost:9999/v1/orders', params=params)
    if resp.ok:
        order_id = resp.json()['order_id']
        print(f'SUCCESS: {action} {qty} {ticker} @ {price}, ID: {order_id}')
        return order_id
    else:
        print(f'FAILED: {resp.text}')
        return None


# # Get all open orders
# resp = s.get('http://localhost:9999/v1/orders')
# if resp.ok:
#     orders = resp.json()
#     print(f"Number of open orders: {len(orders)}")
#     print(f"Orders: {orders}")

def cancel_all():
    resp = s.post('http://localhost:9999/v1/commands/cancel', params={'all': 1})
    if resp.ok:
        result = resp.json()
        print(f'Cancel response: {result}')
        return True
    else:
        print(f'Cancel failed: {resp.text}')
        return False

def get_news():
    resp = s.get('http://localhost:9999/v1/news')  # Uses s.get() inside
    if resp.ok:
        return resp.json()
    return []


def get_market_info(ticker):
    """Get best bid, best ask, last trade"""
    resp = s.get('http://localhost:9999/v1/securities/book', 
                 params={'ticker': ticker})
    if resp.ok:
        book = resp.json()
        best_bid = book['bids'][0]['price'] if book['bids'] else None
        best_ask = book['asks'][0]['price'] if book['asks'] else None
        
        if best_bid and best_ask:
            mid = (best_bid + best_ask) / 2
        else:
            mid = 500  # Default guess
            
        return best_bid, best_ask, mid
    return None, None, 500

In [29]:
def get_my_orders():
    """Get OUR open orders"""
    resp = s.get('http://localhost:9999/v1/orders', params={'ticker': 'APPL'})
    if resp.ok:
        return resp.json()
    return []

def bot_scalper():
    """Smart scalping - only tighten if we're NOT already best"""
    while True:
        tick = get_tick()
        if tick <= 5 or tick > 895:
            break
        
        best_bid, best_ask, mid = get_market_info('APPL')
        my_orders = get_my_orders()
        
        # Check if WE have the best prices already
        my_best_bid = None
        my_best_ask = None
        for order in my_orders:
            if order['action'] == 'BUY' and order['status'] == 'OPEN':
                my_best_bid = order['price']
            if order['action'] == 'SELL' and order['status'] == 'OPEN':
                my_best_ask = order['price']
        
        # Only update if we're NOT the best
        need_update = False
        
        if best_bid and (my_best_bid is None or my_best_bid < best_bid):
            # Someone has better bid than us, join or improve
            my_bid = best_bid
            need_update = True
        elif my_best_bid:
            my_bid = my_best_bid
        else:
            my_bid = mid - 5
        
        if best_ask and (my_best_ask is None or my_best_ask > best_ask):
            # Someone has better ask than us, join or improve
            my_ask = best_ask
            need_update = True
        elif my_best_ask:
            my_ask = my_best_ask
        else:
            my_ask = mid + 5
        
        # Only cancel and repost if needed
        if need_update or len(my_orders) == 0:
            cancel_all()
            send_order('APPL', 'BUY', my_bid, 200)
            send_order('APPL', 'SELL', my_ask, 200)
            print(f'Scalper: Bid {my_bid} | Ask {my_ask}')
        
        time.sleep(0.5)  # Check twice per second

In [39]:
def harvest_bomb():
    """Trade on harvest with RISK MANAGEMENT"""
    # Wait for tick 180
    while get_tick() < 180:
        time.sleep(1)
    
    time.sleep(0.5)  # Let news settle
    
    # Get harvest from news
    news = get_news()
    my_harvest = None
    
    for item in news:
        headline = item.get('headline', '')
        if 'personal harvest of apples' in headline.lower():
            # Parse: "Your personal harvest of apples is 87."
            words = headline.split()
            for i, word in enumerate(words):
                if word == 'is' and i + 1 < len(words):
                    try:
                        my_harvest = int(words[i + 1].replace('.', ''))
                        break
                    except:
                        pass
    
    if my_harvest is None:
        print("ERROR: Could not parse harvest!")
        return
    
    print(f'\n{"="*50}')
    print(f'YOUR HARVEST: {my_harvest}')
    print(f'{"="*50}\n')
    
    market_mid = get_market_info('APPL')[2]
    pos = get_position('APPL')
    
    # RISK MANAGEMENT based on distance from modes
    # Bimodal at 20 and 80, so:
    # 87 = VERY HIGH (7 above high mode)
    # 13 = VERY LOW (7 below low mode)
    
    if my_harvest >= 80:
        # High mode - market thinks 50, true is ~80
        # Edge = +30 per contract, but don't get too big
        distance_from_mode = my_harvest - 80
        confidence = min(distance_from_mode / 10, 1.0)  # Cap at 100%
        
        size = int(300 + 200 * confidence)  # 300-500 contracts
        price = market_mid + 20  # Aggressive but not crazy
        
        cancel_all()
        send_order('APPL', 'BUY', price, size)
        print(f'HIGH HARVEST ({my_harvest}): Buying {size} @ {price}')
        print(f'Confidence: {confidence*100:.0f}%')
        
    elif my_harvest <= 20:
        # Low mode - market thinks 50, true is ~20
        distance_from_mode = 20 - my_harvest
        confidence = min(distance_from_mode / 10, 1.0)
        
        size = int(300 + 200 * confidence)  # 300-500 contracts
        price = market_mid - 20
        
        cancel_all()
        send_order('APPL', 'SELL', price, size)
        print(f'LOW HARVEST ({my_harvest}): Selling {size} @ {price}')
        print(f'Confidence: {confidence*100:.0f}%')
        
    else:
        # Middle range (20-80) - less clear, smaller position
        if my_harvest > 50:
            size = 100
            cancel_all()
            send_order('APPL', 'BUY', market_mid + 10, size)
            print(f'MODERATE HARVEST ({my_harvest}): Small buy {size}')
        else:
            size = 100
            cancel_all()
            send_order('APPL', 'SELL', market_mid - 10, size)
            print(f'MODERATE HARVEST ({my_harvest}): Small sell {size}')
    
    # Monitor position and add if confident
    time.sleep(5)
    new_pos = get_position('APPL')
    print(f'Position after harvest trade: {new_pos}')

In [38]:
print('='*60)
print('APPL DUAL STRATEGY - LIVE')
print('='*60)

harvest_traded = False
last_tick = 0

while True:
    tick = get_tick()
    
    # Stop conditions
    if tick <= 5:
        time.sleep(1)
        continue
    if tick > 895:
        print('\nCase ending...')
        break
    
    pos = get_position('APPL')
    
    # ==================================================
    # STRATEGY 1: BOT SCALPER (Continuous)
    # ==================================================
    if tick != last_tick:
        best_bid, best_ask, mid = get_market_info('APPL')
        my_orders = get_my_orders()
        
        # Check our current orders
        my_best_bid = None
        my_best_ask = None
        for order in my_orders:
            if order['action'] == 'BUY' and order['status'] == 'OPEN':
                my_best_bid = order['price']
            if order['action'] == 'SELL' and order['status'] == 'OPEN':
                my_best_ask = order['price']
        
        # Only update if we're NOT already the best
        need_update = False
        
        if best_bid and (my_best_bid is None or my_best_bid < best_bid):
            my_bid = best_bid  # Join best bid
            need_update = True
        elif my_best_bid:
            my_bid = my_best_bid
        else:
            my_bid = mid - 5
        
        if best_ask and (my_best_ask is None or my_best_ask > best_ask):
            my_ask = best_ask  # Join best ask
            need_update = True
        elif my_best_ask:
            my_ask = my_best_ask
        else:
            my_ask = mid + 5
        
        # Update if needed
        if need_update or len(my_orders) == 0:
            cancel_all()
            send_order('APPL', 'BUY', my_bid, 200)
            send_order('APPL', 'SELL', my_ask, 200)
        
        print(f'Tick {tick} | Pos: {pos} | Bid: {my_bid} | Ask: {my_ask}')
        last_tick = tick
    
    # ==================================================
    # STRATEGY 2: HARVEST BOMB (Tick 180+)
    # ==================================================
    if tick >= 180 and not harvest_traded:
        print('\n' + '='*60)
        print('TICK 180 - HARVEST NEWS!')
        print('='*60)
        
        time.sleep(0.5)
        news = get_news()
        my_harvest = None
        
        for item in news:
            headline = item.get('headline', '')
            if 'personal harvest of apples' in headline.lower():
                words = headline.split()
                for i, word in enumerate(words):
                    if word == 'is' and i + 1 < len(words):
                        try:
                            my_harvest = int(words[i + 1].replace('.', ''))
                            break
                        except:
                            pass
        
        if my_harvest:
            print(f'\n YOUR HARVEST: {my_harvest} \n')
            
            market_mid = get_market_info('APPL')[2]
            
            # Risk-managed sizing
            if my_harvest >= 80:
                distance = my_harvest - 80
                confidence = min(distance / 10, 1.0)
                size = int(300 + 200 * confidence)
                price = market_mid + 20
                
                cancel_all()
                send_order('APPL', 'BUY', price, size)
                print(f'HIGH HARVEST: Buying {size} @ {price}')
                print(f'   Confidence: {confidence*100:.0f}%\n')
                
            elif my_harvest <= 20:
                distance = 20 - my_harvest
                confidence = min(distance / 10, 1.0)
                size = int(300 + 200 * confidence)
                price = market_mid - 20
                
                cancel_all()
                send_order('APPL', 'SELL', price, size)
                print(f'LOW HARVEST: Selling {size} @ {price}')
                print(f'Confidence: {confidence*100:.0f}%\n')
                
            else:
                if my_harvest > 50:
                    size = 100
                    price = market_mid + 10
                    cancel_all()
                    send_order('APPL', 'BUY', price, size)
                    print(f'MODERATE: Small buy {size} @ {price}\n')
                else:
                    size = 100
                    price = market_mid - 10
                    cancel_all()
                    send_order('APPL', 'SELL', price, size)
                    print(f'MODERATE: Small sell {size} @ {price}\n')
            
            harvest_traded = True
            time.sleep(2)
            print(f'Position after harvest trade: {get_position("APPL")}\n')
            print('='*60)
    
    time.sleep(0.5)

# Final cleanup
cancel_all()
final_pos = get_position('APPL')
print(f'\n{"="*60}')
print(f'FINAL POSITION: {final_pos}')
print('DONE!')
print('='*60)


APPL DUAL STRATEGY - LIVE
Cancel response: {'cancelled_order_ids': [147]}
SUCCESS: BUY 200 APPL @ 11.0, ID: 1173
SUCCESS: SELL 200 APPL @ 1009.99, ID: 1174
Tick 55 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 56 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 57 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 58 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 59 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 60 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 61 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 62 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 63 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 64 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 65 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 66 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 67 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 68 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 69 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 70 | Pos: -1000.0 | Bid: 11.0 | Ask: 1009.99
Tick 71 | Pos: -1000.0 | Bid: 11.0 | Ask: 10

KeyboardInterrupt: 